[View in Colaboratory](https://colab.research.google.com/github/thayumaanavan/Self-Driving-Car/blob/master/Self_Driving_Car.ipynb)

In [0]:
#To read data from gdrive
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [57]:
#Download data from gdrive
fileID = drive.CreateFile({'id': '1cBg-zMECiMh0o3mjD8x4yKS3FlQlUPPa'})
print(fileID['title'])
fileID.GetContentFile('IMG.zip') 

IMG.zip


In [1]:
#remove folder if exists already
!rm -r Self-Driving-Car
!ls

datalab  IMG.zip


In [2]:
#clone the repo
!git clone https://github.com/thayumaanavan/Self-Driving-Car.git

Cloning into 'Self-Driving-Car'...
remote: Counting objects: 31, done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 31 (delta 12), reused 20 (delta 6), pack-reused 0
Unpacking objects: 100% (31/31), done.


In [3]:
#extract the data to the respective folder
!ls
import zipfile
zip_ref = zipfile.ZipFile("IMG.zip", 'r')
zip_ref.extractall('Self-Driving-Car/data')
zip_ref.close()

datalab  IMG.zip  Self-Driving-Car


In [4]:
#verify the data is extracted properly
!ls Self-Driving-Car/data

driving_log.csv  IMG


In [0]:
import sys
sys.path.append('Self-Driving-Car')

In [6]:
import pandas as pd # data analysis toolkit - create, read, update, delete datasets
import numpy as np #matrix math
from sklearn.model_selection import train_test_split #to split out training and testing data 
#keras is a high level wrapper on top of tensorflow (machine learning library)
#The Sequential container is a linear stack of layers
from keras.models import Sequential
#popular optimization strategy that uses gradient descent 
from keras.optimizers import Adam
#to save our model periodically as checkpoints for loading later
from keras.callbacks import ModelCheckpoint
#what types of layers do we want our model to have?
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten
#helper class to define input shape and generate training images given image paths & steering angles
from utils import INPUT_SHAPE, batch_generator
#for command line arguments
import argparse
#for reading files
import os

#for debugging, allows for reproducible (deterministic) results 
np.random.seed(0)

Using TensorFlow backend.


In [0]:
def load_data(args):
  
  #reads CSV files into single data frame var
  data_df = pd.read_csv(os.path.join(os.getcwd(), args.data_dir, 'driving_log.csv'), names =['center','left','right','steering','throttle','reverse','speed'])
  
  #Get input & output
  X = data_df[['center','left','right']].values
  y = data_df['steering'].values
  
  #split the data into training,test & validation set
  X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=args.test_size, random_state=0)
  
  return X_train, X_valid, y_train, y_valid

def build_model(args):
  """
  NVIDIA model used
  Image normalization to avoid saturation and make gradients work better.
  Convolution: 5x5, filter: 24, strides: 2x2, activation: ELU
  Convolution: 5x5, filter: 36, strides: 2x2, activation: ELU
  Convolution: 5x5, filter: 48, strides: 2x2, activation: ELU
  Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
  Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
  Drop out (0.5)
  Fully connected: neurons: 100, activation: ELU
  Fully connected: neurons: 50, activation: ELU
  Fully connected: neurons: 10, activation: ELU
  Fully connected: neurons: 1 (output)
  """
  
  model = Sequential()
  model.add(Lambda(lambda x: x/127.5-1.0, input_shape=INPUT_SHAPE))
  model.add(Conv2D(24, (5, 5), activation='elu', strides=(2, 2)))
  model.add(Conv2D(36, (5, 5), activation='elu', strides=(2, 2)))
  model.add(Conv2D(48, (5, 5), activation='elu', strides=(2, 2)))
  model.add(Conv2D(64, (3, 3), activation='elu'))
  model.add(Conv2D(64, (3, 3), activation='elu'))
  model.add(Dropout(args.keep_prob))
  model.add(Flatten())
  model.add(Dense(100, activation='elu'))
  model.add(Dense(50, activation='elu'))
  model.add(Dense(10, activation='elu'))
  model.add(Dense(1))
  model.summary()
  
  return model
  

def train_model(model, args, X_train, X_valid, y_train, y_valid):
  
  checkpoint = ModelCheckpoint('model-{epoch:03d}.h5', 
                              monitor='val_loss',
                              verbose=0,
                              save_best_only=args.save_best_only,
                              mode='auto')
  
  model.compile(loss='mean_squared_error', optimizer=Adam(lr=args.learning_rate))
  
  model.fit_generator(batch_generator(args.data_dir,X_train,y_train, args.batch_size, True), 
                     args.samples_per_epoch,
                     args.nb_epoch,
                     max_q_size=1,
                     validation_data=batch_generator(args.data_dir, X_valid, y_valid, args.batch_size, False),
                     nb_val_samples= len(X_valid),
                     callbacks=[checkpoint],
                     verbose=1)
  

#for command line args
def s2b(s):
    """
    Converts a string to boolean value
    """
    s = s.lower()
    return s == 'true' or s == 'yes' or s == 'y' or s == '1'

In [0]:
def main():
  
  parser = argparse.ArgumentParser(description='Behavioral Cloning Training Program')
  parser.add_argument('-d', help='data directory',        dest='data_dir',          type=str,   default='Self-Driving-Car/data')
  parser.add_argument('-t', help='test size fraction',    dest='test_size',         type=float, default=0.2)
  parser.add_argument('-k', help='drop out probability',  dest='keep_prob',         type=float, default=0.5)
  parser.add_argument('-n', help='number of epochs',      dest='nb_epoch',          type=int,   default=10)
  parser.add_argument('-s', help='samples per epoch',     dest='samples_per_epoch', type=int,   default=20000)
  parser.add_argument('-b', help='batch size',            dest='batch_size',        type=int,   default=40)
  parser.add_argument('-o', help='save best models only', dest='save_best_only',    type=s2b,   default='true')
  parser.add_argument('-l', help='learning rate',         dest='learning_rate',     type=float, default=1.0e-4)
  args = parser.parse_args()

  #print parameters
  print('-' * 30)
  print('Parameters')
  print('-' * 30)
  for key, value in vars(args).items():
      print('{:<20} := {}'.format(key, value))
  print('-' * 30)

  #load data
  data = load_data(args)
  
  #build model
  model = build_model(args)
  
  #train_model
  train_model(model,args, *data)

In [0]:
sys.argv=['']
main()
del sys

------------------------------
Parameters
------------------------------
data_dir             := Self-Driving-Car/data
test_size            := 0.2
keep_prob            := 0.5
nb_epoch             := 10
samples_per_epoch    := 20000
batch_size           := 40
save_best_only       := True
learning_rate        := 0.0001
------------------------------
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 66, 200, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 22, 48)         43248     
________________________________________________________

__main__:66: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
__main__:66: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., 20000, 10, validation_data=<generator..., callbacks=[<keras.ca..., verbose=1, validation_steps=233, max_queue_size=1)`


Epoch 1/10
 2940/20000 [===>..........................] - ETA: 46:09 - loss: 0.0921